In [ ]:
!pip install pyngrok
!pip install streamlit
!pip install flask_ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
!ngrok config add-authtoken 2uB8e9DPAgoYWwtVAyx9gfNAvOn_5UQ8G3eGTHUhhWhHDen9W

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify, render_template_string
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
from pyngrok import ngrok

app = Flask(__name__)

# Load the trained model
MODEL_PATH = "/content/Model_InceptionV3 (2).h5"
model = load_model(MODEL_PATH)

# Define constants
IMAGE_RES = 224
class_names = [
    "Alovera", "Amla", "Bamboo", "Beans", "Castor", "Catharanthus",
    "Coffee", "Coriender", "Globe Amarnath", "Jackfruit", "Kasambruga",
    "Lemon", "Mango", "Marigold", "Palak"
]

plant_info = {
    "Alovera": {
        "characteristics": "Rich in vitamins C and E; contains antioxidants and anti-inflammatory compounds.",
        "medicinal_uses": [
            "Used in Ayurveda, Unani, and Chinese medicine for skincare and digestion.",
            "Used in cosmetics and ointments for wound healing.",
            "Used in herbal tonics for liver detox."
        ],
        "famous_in": "Tropical and subtropical regions.",
        "how_to_use": "Extract aloe vera gel and apply it directly to the skin for hydration or mix with honey for a soothing drink."
    },
    "Amla": {
        "characteristics": "High in vitamin C and antioxidants; rich in fiber and polyphenols.",
        "medicinal_uses": [
            "Used in Ayurveda (Chyawanprash, Triphala) for immunity and digestion.",
            "Used in herbal tonics for improving eyesight and hair health.",
            "Used in traditional medicine for diabetes control."
        ],
        "famous_in": "India and Southeast Asia.",
        "how_to_use": "Consume raw, as juice, or in powder form mixed with honey or warm water."
    },
    "Bamboo": {
        "characteristics": "Rich in silica; contains antioxidants and antibacterial properties.",
        "medicinal_uses": [
            "Used in traditional medicine for treating respiratory disorders.",
            "Bamboo shoots are used for digestive health and weight management.",
            "Used in Ayurvedic and Chinese medicine for bone strengthening."
        ],
        "famous_in": "Asia, South America, and Africa.",
        "how_to_use": "Consume bamboo shoots in soups, curries, or stir-fried dishes after proper boiling."
    },
    "Beans": {
        "characteristics": "High in protein, fiber, and essential vitamins; contains folate and iron.",
        "medicinal_uses": [
            "Helps in regulating blood sugar levels.",
            "Rich in fiber, aiding in digestion and heart health.",
            "Used in traditional medicine for improving kidney function."
        ],
        "famous_in": "Worldwide, especially in Asia and South America.",
        "how_to_use": "Cooked in curries, soups, or salads; sprouted beans are great for digestion."
    },
    "Castor": {
        "characteristics": "Rich in ricinoleic acid; has antifungal and anti-inflammatory properties.",
        "medicinal_uses": [
            "Used in Ayurveda for joint pain relief and detoxification.",
            "Castor oil is used for hair growth and skincare.",
            "Used in herbal treatments for constipation."
        ],
        "famous_in": "India, Africa, and the Mediterranean.",
        "how_to_use": "Apply castor oil to the scalp for hair growth or use a few drops as a laxative (under medical supervision)."
    },
    "Catharanthus": {
        "characteristics": "Contains alkaloids with anticancer properties; used in chemotherapy drugs.",
        "medicinal_uses": [
            "Used in modern medicine for treating leukemia and other cancers.",
            "Traditionally used for diabetes management.",
            "Used in folk medicine for treating high blood pressure."
        ],
        "famous_in": "Tropical and subtropical regions, especially Madagascar.",
        "how_to_use": "Used in herbal decoctions or as an extract in pharmaceuticals (not for self-medication)."
    },
    "Coffee": {
        "characteristics": "Rich in caffeine and antioxidants; contains polyphenols and chlorogenic acid.",
        "medicinal_uses": [
            "Improves cognitive function and alertness.",
            "Used in herbal medicine for weight loss and metabolism boosting.",
            "Contains anti-inflammatory compounds beneficial for liver health."
        ],
        "famous_in": "South America, Africa, and Southeast Asia.",
        "how_to_use": "Brewed as a beverage; can be used in skincare products for exfoliation."
    },
    "Coriender": {
        "characteristics": "Rich in vitamins A, C, and K; contains antioxidants and essential oils.",
        "medicinal_uses": [
            "Used in Ayurveda for digestion and detoxification.",
            "Helps in managing cholesterol levels and blood sugar.",
            "Used in traditional remedies for skin health."
        ],
        "famous_in": "India, the Mediterranean, and Central America.",
        "how_to_use": "Used as a spice in cooking; coriander water is used for detoxification."
    },
    "Globe Amarnath": {
        "characteristics": "Rich in betacyanins; contains anti-inflammatory and antimicrobial properties.",
        "medicinal_uses": [
            "Used in traditional medicine for treating respiratory issues.",
            "Used in herbal remedies for reducing blood pressure.",
            "Applied as a poultice for wound healing."
        ],
        "famous_in": "Tropical and subtropical regions.",
        "how_to_use": "Flowers are brewed into tea or used in herbal tonics."
    },
    "Jackfruit": {
        "characteristics": "High in vitamin C, fiber, and antioxidants; contains flavonoids and carotenoids.",
        "medicinal_uses": [
            "Used in Ayurvedic medicine for boosting immunity.",
            "Jackfruit seeds are used in traditional treatments for digestion.",
            "Helps in managing blood sugar levels."
        ],
        "famous_in": "South and Southeast Asia.",
        "how_to_use": "Consumed as a fruit, cooked as a vegetable, or ground into flour."
    },
    "Kasambruga": {
        "characteristics": "Contains flavonoids and saponins with antimicrobial properties.",
        "medicinal_uses": [
            "Used in traditional medicine for fever and inflammation.",
            "Helps in treating skin infections and wounds.",
            "Used in Ayurvedic medicine for digestive disorders."
        ],
        "famous_in": "India and Sri Lanka.",
        "how_to_use": "Used in herbal infusions or applied as a paste for skin ailments."
    },
    "Lemon": {
        "characteristics": "Rich in vitamin C, flavonoids, and antioxidants; has antibacterial properties.",
        "medicinal_uses": [
            "Used in natural remedies for immunity boosting and detoxification.",
            "Used in herbal treatments for kidney stones and digestion.",
            "Applied topically for skin brightening."
        ],
        "famous_in": "Worldwide, especially in Mediterranean and tropical regions.",
        "how_to_use": "Consumed as juice, in teas, or applied topically for skincare."
    },
    "Mango": {
        "characteristics": "High in vitamins A and C; contains polyphenols and dietary fiber.",
        "medicinal_uses": [
            "Used in traditional medicine for digestion and immunity boosting.",
            "Mango leaves are used for managing diabetes.",
            "Mango kernel extract is used for skincare."
        ],
        "famous_in": "India, Southeast Asia, and South America.",
        "how_to_use": "Eaten as a fruit, juice, or dried; leaves are used in herbal teas."
    },
    "Marigold": {
        "characteristics": "Rich in lutein and flavonoids; has anti-inflammatory and antiseptic properties.",
        "medicinal_uses": [
            "Used in herbal medicine for wound healing and skin infections.",
            "Used in Ayurvedic treatments for eye health.",
            "Marigold tea is used for digestive disorders."
        ],
        "famous_in": "Worldwide, especially in Europe and India.",
        "how_to_use": "Used as an herbal tea or in skincare ointments."
    },
    "Palak": {
        "characteristics": "High in iron, calcium, and vitamins A and C; contains folate and antioxidants.",
        "medicinal_uses": [
            "Used in herbal medicine for anemia and bone health.",
            "Rich in fiber, aiding in digestion and weight management.",
            "Used in Ayurveda for liver health and detoxification."
        ],
        "famous_in": "Worldwide, especially in India and the Mediterranean.",
        "how_to_use": "Consumed as a cooked vegetable, in smoothies, or raw in salads."
    }
}


@app.route("/")
def home():
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Plant Image Classification</title>
        <style>
            body { font-family: 'Arial', sans-serif; background-color: #f1f7f3; text-align: center; padding: 20px; }
            h1 { color: #3c763d; }
            input, button { margin-top: 10px; padding: 10px; font-size: 1em; }
            #result, #characteristics, #uses, #famous-in, #how-to-use, #accuracy { margin-top: 15px; font-size: 1.1em; }
            img { max-width: 200px; margin-top: 10px; border: 2px solid #7bbf6a; }
        </style>
    </head>
    <body>
        <h1>Deep Learning based Robust Medicinal Leaves Identification</h1>
        <form id="upload-form" enctype="multipart/form-data">
            <input type="file" id="image-input" accept="image/*" required>
            <button type="submit">Classify Image</button>
        </form>
        <div id="result"></div>
        <div id="accuracy"></div>
        <div id="characteristics"></div>
        <div id="uses"></div>
        <div id="famous-in"></div>
        <div id="how-to-use"></div>
        <img id="preview" src="" alt="" style="display: none;">

        <script>
            const form = document.getElementById("upload-form");
            const imageInput = document.getElementById("image-input");
            const resultDiv = document.getElementById("result");
            const characteristicsDiv = document.getElementById("characteristics");
            const usesDiv = document.getElementById("uses");
            const famousInDiv = document.getElementById("famous-in");
            const howToUseDiv = document.getElementById("how-to-use");
            const accuracyDiv = document.getElementById("accuracy");
            const previewImg = document.getElementById("preview");

            form.addEventListener("submit", async (event) => {
                event.preventDefault();
                const file = imageInput.files[0];
                const reader = new FileReader();
                reader.onload = () => {
                    previewImg.src = reader.result;
                    previewImg.style.display = "block";
                };
                reader.readAsDataURL(file);
                const formData = new FormData();
                formData.append("file", file);
                resultDiv.textContent = "Classifying...";

                try {
                    const response = await fetch("/predict", { method: "POST", body: formData });
                    const data = await response.json();
                    resultDiv.textContent = `Predicted Class: ${data.predicted_class}`;
                    accuracyDiv.textContent = `Accuracy: ${(data.accuracy * 100).toFixed(2)}%`;
                    characteristicsDiv.textContent = `Characteristics: ${data.characteristics}`;
                    usesDiv.textContent = `Uses: ${data.uses.join(", ")}`;
                    famousInDiv.textContent = `Famous in: ${data.famous_in}`;
                    howToUseDiv.textContent = `How to use: ${data.how_to_use}`;
                } catch (error) {
                    resultDiv.textContent = "Error: " + error.message;
                }
            });
        </script>
    </body>
    </html>
    """)

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400
    file = request.files["file"]
    if file.filename == "":
        return jsonify({"error": "No file selected"}), 400
    temp_path = os.path.join("uploads", file.filename)
    os.makedirs("uploads", exist_ok=True)
    file.save(temp_path)

    input_image = load_img(temp_path, target_size=(IMAGE_RES, IMAGE_RES))
    input_image_array = img_to_array(input_image) / 255.0
    input_image_array = np.expand_dims(input_image_array, axis=0)

    predictions = model.predict(input_image_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_names[predicted_class_index]
    accuracy = np.max(predictions)

    plant_details = plant_info.get(predicted_class_name, {
        "characteristics": "N/A", "medicinal_uses": ["N/A"], "famous_in": "N/A", "how_to_use": "N/A"
    })

    os.remove(temp_path)
    return jsonify({
        "predicted_class": predicted_class_name,
        "characteristics": plant_details["characteristics"],
        "uses": plant_details["medicinal_uses"],
        "famous_in": plant_details["famous_in"],
        "how_to_use": plant_details["how_to_use"],
        "accuracy": float(accuracy)
    })

if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run(port=5000)
